# Data Preprocessing

Link to dataset: https://www.kaggle.com/sid321axn/amazon-alexa-reviews

In [1]:
# global imports
# pyspark
import pyspark
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as W
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, ArrayType
# time
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.types import TimestampType
# text transformations
import contractions
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from nltk.stem import WordNetLemmatizer

spark = SparkSession.builder.getOrCreate()

# Examine Dataset

In [2]:
# import data
df = spark.read.option('delimiter','\t').csv('amazon_alexa.tsv',header=True,inferSchema=True)
df.show(2)

+------+---------+----------------+----------------+--------+
|rating|     date|       variation|verified_reviews|feedback|
+------+---------+----------------+----------------+--------+
|     5|31-Jul-18|Charcoal Fabric |   Love my Echo!|       1|
|     5|31-Jul-18|Charcoal Fabric |       Loved it!|       1|
+------+---------+----------------+----------------+--------+
only showing top 2 rows



In [3]:
# output dimensions of the dataset
print("Number of rows: ",df.count())
print("Number of columns: ",len(df.columns))

Number of rows:  3150
Number of columns:  5


In [4]:
# output data types of each column
df.dtypes

[('rating', 'int'),
 ('date', 'string'),
 ('variation', 'string'),
 ('verified_reviews', 'string'),
 ('feedback', 'int')]

In [5]:
# determine what the variable 'feedback' means
df.where(W.col('feedback') == 0).select('rating').distinct().show()
df.where(W.col('feedback') == 1).select('rating').distinct().show()

+------+
|rating|
+------+
|     1|
|     2|
+------+

+------+
|rating|
+------+
|     3|
|     5|
|     4|
+------+



# Data Cleaning

## Select Columns

`feedback` is used instead of `rating` to create a binary classification problem.

In [6]:
# select columns for further study
df = df.drop('rating')
# confirm it worked
df.show(2)

+---------+----------------+----------------+--------+
|     date|       variation|verified_reviews|feedback|
+---------+----------------+----------------+--------+
|31-Jul-18|Charcoal Fabric |   Love my Echo!|       1|
|31-Jul-18|Charcoal Fabric |       Loved it!|       1|
+---------+----------------+----------------+--------+
only showing top 2 rows



## Missing Values

The dataset did not have missing values.

In [7]:
# number of missing values in each column
df.select(*(W.sum(W.col(c).isNull().cast('int')).alias(c) for c in df.columns)).show()

+----+---------+----------------+--------+
|date|variation|verified_reviews|feedback|
+----+---------+----------------+--------+
|   0|        0|               0|       0|
+----+---------+----------------+--------+



## Duplicates

I chose to handle duplicates at a later stage, after I apply transformations to the dataset to extract more informtion from the `verified_reviews` column. Short reviews, like "Love it," are probably unique values. However, longer reviews, like "very handy in the kitchen, sets timer and gives me extra help on setting oven temps," were probably mistakenly entered twice.

In [8]:
# number of distinct observations
df.distinct().count()

2427

In [9]:
# output rows that are duplicates
df.groupBy('date','variation','verified_reviews','feedback')\
  .count()\
  .where(W.col('count')>1)\
  .show()

+---------+--------------------+--------------------+--------+-----+
|     date|           variation|    verified_reviews|feedback|count|
+---------+--------------------+--------------------+--------+-----+
|30-Jul-18|   Sandstone Fabric |Love my Echo. Sti...|       1|    2|
|30-Jul-18|Heather Gray Fabric |so far love it wo...|       1|    2|
|29-Jul-18|    Charcoal Fabric |Got a good deal a...|       1|    2|
|30-Jul-18|          Black  Dot|Love it!! Don’t k...|       1|    2|
|28-Jul-18|    Charcoal Fabric |      Works awesome!|       1|    2|
|30-Jul-18|          Black  Dot|This worked well ...|       0|    2|
|30-Jul-18|          White  Dot|Alexa seems to ge...|       1|    2|
|30-Jul-18|          White  Dot|I was hoping the ...|       1|    2|
|30-Jul-18|    Charcoal Fabric |I love having an ...|       1|    2|
|29-Jul-18|    Charcoal Fabric |This was given to...|       1|    2|
|31-Jul-18|          Black  Dot|                    |       1|    2|
|30-Jul-18|          Black  Dot|Go

## `variation` Variable

All levels in `variation` were properly entered.

In [10]:
# confirm all observations were 
df.select('variation').distinct().show(truncate=False)

+----------------------------+
|variation                   |
+----------------------------+
|Heather Gray Fabric         |
|Black  Dot                  |
|Oak Finish                  |
|Configuration: Fire TV Stick|
|Sandstone Fabric            |
|White  Show                 |
|White  Plus                 |
|White  Spot                 |
|Black  Spot                 |
|Black  Show                 |
|Walnut Finish               |
|White                       |
|Charcoal Fabric             |
|White  Dot                  |
|Black  Plus                 |
|Black                       |
+----------------------------+



## `date` Variable

Convert `date` from `StringType` to `DateType`.

In [11]:
# convert date variable from string to date type
df = df.withColumnRenamed('date','date2')
# convert to timestamp type
df = df.withColumn('date',from_unixtime(unix_timestamp('date2','dd-MMM-yy')))
df = df.withColumn('date',df['date'].cast(TimestampType()))
# remove the old date column
df = df.drop('date2')
df.show(2)

+----------------+----------------+--------+-------------------+
|       variation|verified_reviews|feedback|               date|
+----------------+----------------+--------+-------------------+
|Charcoal Fabric |   Love my Echo!|       1|2018-07-31 00:00:00|
|Charcoal Fabric |       Loved it!|       1|2018-07-31 00:00:00|
+----------------+----------------+--------+-------------------+
only showing top 2 rows



## `feedback` Variable

Since this is an imbalanced classification problem, change feedback variable values so the majority class is class `0`.

In [12]:
# show that it is an imbalanced classification problem
df.groupby('feedback').count().show()

+--------+-----+
|feedback|count|
+--------+-----+
|       1| 2893|
|       0|  257|
+--------+-----+



In [13]:
# create dictionary
feedback_dict = {0:1,1:0}
# create a mapping function
map_func = W.udf(lambda x: feedback_dict[x])
# convert values
df = df.withColumn('label',map_func('feedback').cast(IntegerType()))
# output data frame
df.select('feedback','label').show(2)

+--------+-----+
|feedback|label|
+--------+-----+
|       1|    0|
|       1|    0|
+--------+-----+
only showing top 2 rows



In [14]:
# confirm label worked correctly
df.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  257|
|    0| 2893|
+-----+-----+



# Feature Engineering

Create `length` variable

## Expand Contractions

In [15]:
# function for expanding contractions
def fix_contractions(text):
    return contractions.fix(text)
# udf for expanding contractions
contractions_udf = W.udf(lambda row: fix_contractions(row) , StringType())
# add column with contractions expanded
df = df.withColumn('text_clean', contractions_udf('verified_reviews'))
# output data frame
df.show(2)

+----------------+----------------+--------+-------------------+-----+-------------+
|       variation|verified_reviews|feedback|               date|label|   text_clean|
+----------------+----------------+--------+-------------------+-----+-------------+
|Charcoal Fabric |   Love my Echo!|       1|2018-07-31 00:00:00|    0|Love my Echo!|
|Charcoal Fabric |       Loved it!|       1|2018-07-31 00:00:00|    0|    Loved it!|
+----------------+----------------+--------+-------------------+-----+-------------+
only showing top 2 rows



## Tokenize the Text

In [16]:
# tokenize the text
rt = RegexTokenizer().setInputCol('text_clean')\
                     .setOutputCol('text_vec')\
                     .setPattern('\s+|[\W]')\
                     .setToLowercase(True)
# transform data
df = rt.transform(df)
# output dataframe
df.show(2)

+----------------+----------------+--------+-------------------+-----+-------------+----------------+
|       variation|verified_reviews|feedback|               date|label|   text_clean|        text_vec|
+----------------+----------------+--------+-------------------+-----+-------------+----------------+
|Charcoal Fabric |   Love my Echo!|       1|2018-07-31 00:00:00|    0|Love my Echo!|[love, my, echo]|
|Charcoal Fabric |       Loved it!|       1|2018-07-31 00:00:00|    0|    Loved it!|     [loved, it]|
+----------------+----------------+--------+-------------------+-----+-------------+----------------+
only showing top 2 rows



## Create `length` Variable

In [17]:
# function for getting the length
def length(text):
    return len(text)
# udf for getting the length
length_udf = W.udf(lambda row: length(row), IntegerType())
# add column with length
df = df.withColumn('length', length_udf('text_vec'))
# output data frame
df.show(2)

+----------------+----------------+--------+-------------------+-----+-------------+----------------+------+
|       variation|verified_reviews|feedback|               date|label|   text_clean|        text_vec|length|
+----------------+----------------+--------+-------------------+-----+-------------+----------------+------+
|Charcoal Fabric |   Love my Echo!|       1|2018-07-31 00:00:00|    0|Love my Echo!|[love, my, echo]|     3|
|Charcoal Fabric |       Loved it!|       1|2018-07-31 00:00:00|    0|    Loved it!|     [loved, it]|     2|
+----------------+----------------+--------+-------------------+-----+-------------+----------------+------+
only showing top 2 rows



In [18]:
# filter the rows where the length is greater than 0
df = df.where(W.col('length')>0)
# count the number of rows
df.count()

3066

# Data Frames for Exploratory Data Analysis and Machine Learning

In [19]:
# create data frame for eda
eda_df = df.select('variation','verified_reviews','date','text_clean','length','label')
# output data frame
eda_df.show(2)

+----------------+----------------+-------------------+-------------+------+-----+
|       variation|verified_reviews|               date|   text_clean|length|label|
+----------------+----------------+-------------------+-------------+------+-----+
|Charcoal Fabric |   Love my Echo!|2018-07-31 00:00:00|Love my Echo!|     3|    0|
|Charcoal Fabric |       Loved it!|2018-07-31 00:00:00|    Loved it!|     2|    0|
+----------------+----------------+-------------------+-------------+------+-----+
only showing top 2 rows



In [20]:
# create data frame for ml
ml_df = df.select('verified_reviews','text_vec','label')
# output data frame
ml_df.show(2)

+----------------+----------------+-----+
|verified_reviews|        text_vec|label|
+----------------+----------------+-----+
|   Love my Echo!|[love, my, echo]|    0|
|       Loved it!|     [loved, it]|    0|
+----------------+----------------+-----+
only showing top 2 rows



## Handle Duplicates

Duplicate rows with `length > 5` were removed in `eda_df`. Duplicate rows were removed `ml_df`.

In [21]:
# number of distinct observations in eda_df
eda_df.distinct().count()

2380

In [22]:
# remove duplicate rows where length>5
eda_df = eda_df.withColumn('temp_id', W.monotonically_increasing_id())
window = Window.partitionBy([W.col('variation'),W.col('verified_reviews'),W.col('label'),W.col('date')])\
               .orderBy('temp_id')
eda_df = eda_df.withColumn('id', W.dense_rank().over(window)).filter((W.col('length')<=5)|(W.col('id')==1))
# drop extra columns
eda_df = eda_df.drop('temp_id','id')
# output data frame
eda_df.show(2)

+---------+--------------------+-------------------+--------------------+------+-----+
|variation|    verified_reviews|               date|          text_clean|length|label|
+---------+--------------------+-------------------+--------------------+------+-----+
|    Black|Love my Alexa Ech...|2018-06-11 00:00:00|Love my Alexa Ech...|    66|    0|
|    Black|Overall good devi...|2018-05-31 00:00:00|Overall good devi...|    32|    0|
+---------+--------------------+-------------------+--------------------+------+-----+
only showing top 2 rows



In [23]:
# confirm duplicates were removed
eda_df.count()

2554

In [24]:
# number of distinct observations in ml_df
ml_df.distinct().count()

2296

In [25]:
# drop duplicates for ml_df
ml_df = ml_df.dropDuplicates()
# confirm that duplicates were dropped
ml_df.count()

2296

# Transform Text Data

## Remove Stopwords

In [26]:
# create english stopwords
english = StopWordsRemover().loadDefaultStopWords('english')
# stopwords transformer
stops = StopWordsRemover().setStopWords(english)\
                          .setInputCol('text_vec')\
                          .setOutputCol('text_vec_stop')
# transform dataframe
ml_df = stops.transform(ml_df)
# output dataframe
ml_df.show(2)

+--------------------+--------------------+-----+--------------------+
|    verified_reviews|            text_vec|label|       text_vec_stop|
+--------------------+--------------------+-----+--------------------+
|Great addition to...|[great, addition,...|    0|[great, addition,...|
|it's great for mu...|[it, is, great, f...|    0|[great, music, li...|
+--------------------+--------------------+-----+--------------------+
only showing top 2 rows



## Lematize Words

In [27]:
# create word lemmatizer object
wnl = WordNetLemmatizer()
# function for lemmatizing words
def wnl_row(row):
    return [wnl.lemmatize(x) for x in row]
# udf for lemmatizing words
lemmatizer_udf = W.udf(lambda row: wnl_row(row) , ArrayType(StringType()))
# create column of lemmatized words
ml_df = ml_df.withColumn('text_vec_lem', lemmatizer_udf('text_vec_stop'))
# output dataframe
ml_df.show(2)

+--------------------+--------------------+-----+--------------------+--------------------+
|    verified_reviews|            text_vec|label|       text_vec_stop|        text_vec_lem|
+--------------------+--------------------+-----+--------------------+--------------------+
|Great addition to...|[great, addition,...|    0|[great, addition,...|[great, addition,...|
|it's great for mu...|[it, is, great, f...|    0|[great, music, li...|[great, music, li...|
+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 2 rows



# Train and Test Sets

70% of the data was used to train the models, 30% of the data was used to test the models. 

In [28]:
# create train and test sets
train,test = ml_df.randomSplit([0.7,0.3],116)

In [29]:
# output counts for train and test sets
print("Number of rows in train set: ",train.count())
print("Number of rows in test set: ",test.count())

Number of rows in train set:  1583
Number of rows in test set:  713


## Vectorize Words

Create a vocabulary of words that appears in 1% of the documents

In [30]:
# create transformer
cv = CountVectorizer().setInputCol('text_vec_lem')\
                      .setOutputCol('count_vec')\
                      .setMinDF(train.count()*0.005)
# fit the transformer
model_cv = cv.fit(train)
# transform the data
train = model_cv.transform(train)
# output the vectorized column
train.select('count_vec').show(2, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|count_vec                                                                                                                                                                                                                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(495,[1,2,43,284],[1.0,1

In [31]:
# fit the vocabulary on the test data
test = model_cv.transform(test)
# output the vectorized column
test.select('count_vec').show(2, truncate=False)

+-----------------------------------------------------------------------------------------------------+
|count_vec                                                                                            |
+-----------------------------------------------------------------------------------------------------+
|(495,[],[])                                                                                          |
|(495,[1,5,6,7,10,11,45,72,123,143,198,266,418],[2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+-----------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [32]:
# create vocabulary
vocab = model_cv.vocabulary

## Create `weights` Variable

In [33]:
# obtain balancing ratio
num_0 = train.select('label').where(W.col('label')==0).count()
total = train.count()
ratio = num_0/total
# create weights column
train = train.withColumn('weights', W.when(train.label==1,ratio).otherwise(1-ratio))
train.show(2)

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+-------------------+
|    verified_reviews|            text_vec|label|       text_vec_stop|        text_vec_lem|           count_vec|            weights|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+-------------------+
|Great addition to...|[great, addition,...|    0|[great, addition,...|[great, addition,...|(495,[1,2,43,284]...|0.08970309538850285|
|I love this littl...|[i, love, this, l...|    0|[love, little, sp...|[love, little, sp...|(495,[0,2,7,8,10,...|0.08970309538850285|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+-------------------+
only showing top 2 rows



## Merge `eda_df` and `train`

In [34]:
# create `eda_df_train` and drop 'label' since it exists in `train`
eda_df_train = eda_df.drop('label')
# join the data frames
eda_df_train = eda_df_train.join(train,on=['verified_reviews'],how='inner')
# output the data frame
eda_df_train.show(2)

+--------------------+-----------+-------------------+--------------------+------+--------------------+-----+--------------------+--------------------+--------------------+-------------------+
|    verified_reviews|  variation|               date|          text_clean|length|            text_vec|label|       text_vec_stop|        text_vec_lem|           count_vec|            weights|
+--------------------+-----------+-------------------+--------------------+------+--------------------+-----+--------------------+--------------------+--------------------+-------------------+
|Love my Alexa Ech...|      Black|2018-06-11 00:00:00|Love my Alexa Ech...|    66|[love, my, alexa,...|    0|[love, alexa, ech...|[love, alexa, ech...|(495,[0,1,2,3,4,7...|0.08970309538850285|
|Not that much dif...|Black  Plus|2018-07-30 00:00:00|Not that much dif...|    58|[not, that, much,...|    0|[much, different,...|[much, different,...|(495,[3,8,15,26,2...|0.08970309538850285|
+--------------------+-----------+-

In [35]:
# output the number of rows
eda_df_train.count()

1771

In [36]:
# select columns
eda_df_train = eda_df_train.select('variation','label','date','text_clean','length')
# output data frame
eda_df_train.show(2)

+-----------+-----+-------------------+--------------------+------+
|  variation|label|               date|          text_clean|length|
+-----------+-----+-------------------+--------------------+------+
|      Black|    0|2018-06-11 00:00:00|Love my Alexa Ech...|    66|
|Black  Plus|    0|2018-07-30 00:00:00|Not that much dif...|    58|
+-----------+-----+-------------------+--------------------+------+
only showing top 2 rows



## Select Columns

In [37]:
# select columns for train
train = train.select('count_vec','label','weights')
# confirm it worked
train.show(2)

+--------------------+-----+-------------------+
|           count_vec|label|            weights|
+--------------------+-----+-------------------+
|(495,[1,2,43,284]...|    0|0.08970309538850285|
|(495,[0,2,7,8,10,...|    0|0.08970309538850285|
+--------------------+-----+-------------------+
only showing top 2 rows



In [38]:
# select columns for train
test = test.select('count_vec','label')
# confirm it worked
test.show(2)

+--------------------+-----+
|           count_vec|label|
+--------------------+-----+
|         (495,[],[])|    0|
|(495,[1,5,6,7,10,...|    0|
+--------------------+-----+
only showing top 2 rows



## Store Data Frames

In [39]:
# store dataframes for additional programs
dfs = [train.toPandas(),test.toPandas(),eda_df_train.toPandas(),vocab]
%store dfs

Stored 'dfs' (list)
